In [1]:
from bs4 import BeautifulSoup
import urllib.request
import re
import mysql.connector
import os
import string


In [2]:
#connect to mysql server
cnx = mysql.connector.connect(host='152.19.68.141', user='ctolson', password='ilaYOU5!', database='sephora_cosmetics')
cursor = cnx.cursor()

#mysql insert functions
add_ingredient = ("INSERT INTO Product_Ingredient "
                    "(product_id, ingredient_name) "
                    "VALUES (%s, %s)")
add_review = ("INSERT INTO Reviews "
                    "(product_id, review, reviewer) "
                    "VALUES (%s, %s, %s)")
add_product = ("INSERT INTO Product "
                       "(name, brand, type, url) "
                       "VALUES (%s, %s, %s, %s)")


In [3]:
search_topics = ['blush','conditioner-hair','mascara','foundation-makeup', 'lipstick']     

for search in search_topics:
    print(search)
    #scrape by search 
    link_main = 'http://www.sephora.com/'
    term = search+'?pageSize=-1'
    req = urllib.request.Request(link_main+term, headers={'User-Agent': 'Firefox'})
    webpage = urllib.request.urlopen(req).read()
    soup_web = BeautifulSoup(webpage, 'html.parser')
    
    #parse product urls
    product_url = soup_web.find_all('script', {'type': 'application/ld+json'})[1]
    product_url = str(product_url)
    product_url = re.findall("(url\":\")(.*?)(\"},{\")", product_url)
    product_url = [x[1] for x in product_url]

    #get product information
    for i in range(0, len(product_url)):
        print(i)
        try:   
            #scrap product page
            req_pg = urllib.request.Request(product_url[i], headers={'User-Agent': 'Chrome'})
            page = urllib.request.urlopen(req_pg).read()
            soup = BeautifulSoup(page, 'html.parser')
    
            #parse ingredients
            ingredients = soup.find_all('script', {'type': 'text/json'})[4]
            ingredients = str(ingredients)
            ingredients = re.sub("(,\"is_new\":true)", "", ingredients)
            ingredients = ingredients.split("ingredients\":")[1]
            ingredients = ingredients.split('","')[0]
            if ingredients == '"':
                continue
            cleanr = re.compile('<.*?>')
            ingredients = re.sub(cleanr, '', ingredients)
            ingredients = ingredients.split(",")
            ingredients = [re.sub(r'[^\w\s]','',x) for x in ingredients]
            ingredients = [re.sub(' r n ','',x) for x in ingredients]
                
            #parse brand name
            brand = soup.find_all('script', {'type': 'text/json'})[4]
            brand = str(brand)
            brand = brand.split("brand_name\":")[1]
            brand = brand.split("},\"sku_number")[0]
            brand = re.split(',', brand)
            brand = str(brand)
            brand = brand[3:-3]
            brand = brand.split('"}')[0] 

            #parse product name
            name = soup.find_all('script', {'type': 'text/json'})[4]
            name = str(name)
            name = name.split("display_name\":")[1]
            name = name.split(",\"variation_type")[0]
            name = re.split(',', name)
            name = str(name)
            name = name[3:-3]
                          
            #parse review page
            keywords = soup.find_all('meta')
            keywords = str(keywords)
            keywords = keywords.split('//reviews.sephora.com/static/')[1]
            keywords = keywords.split('/')[0]
            keywords2 = product_url[i].split('-')
            keywords2 = keywords2[len(keywords2)-1]
            link = 'http://reviews.sephora.com/'+keywords+'/'+keywords2+'/reviews.htm?format=embedded&amp;page=1m&amp;'
            req_pg2 = urllib.request.Request(link, headers={'User-Agent': 'Chrome'})
            page2 = urllib.request.urlopen(req_pg2).read()
            soup2 = BeautifulSoup(page2, 'html.parser')
            rg = soup2.find_all('span', {'class': 'BVRRNumber'})
            if rg == []:
                continue
            
            #insert product into mysql
            try:
                #product table
                data_product = (name, brand, search, str(product_url[i]))
                cursor.execute(add_product, data_product)
                product_id = cursor.lastrowid
                    
                #product-ingredient table
                for ingredient_name in ingredients:
                    data_ingredient = (product_id, ingredient_name)
                    cursor.execute(add_ingredient, data_ingredient)

            except:
                #close mysql server
                cnx.close()
                #connect to mysql server
                cnx = mysql.connector.connect(host='152.19.68.141', user='ctolson', password='ilaYOU5!', database='sephora_cosmetics')
                cursor = cnx.cursor()
                #continue
                continue
                
            rg = str(rg).split("class=\"BVRRNumber\">")[1]
            rg = rg.split("</span")[0]
            rg = re.sub(",", "", rg)
            rg = int(int(rg)/5)
        
            #parse reviews
            for num in range(1,rg):
                link = 'http://reviews.sephora.com/'+keywords+'/'+keywords2+'/reviews.htm?format=embedded&page='+str(num)+'&scrollToTop=true'
                req_pg3 = urllib.request.Request(link, headers={'User-Agent': 'Chrome'})
                page3 = urllib.request.urlopen(req_pg3).read()
                soup3 = BeautifulSoup(page3, 'html.parser')
                text = soup3.find_all('div', {'class': 'BVRRReviewTextContainer'})
                user = soup3.find_all('span', {'class': 'BVRRNickname'})
                for z in range(0,len(text)):
                    review = text[z].find_all('span', {'class': 'BVRRReviewText'})
                    review = str(review).split("class=\"BVRRReviewText\">")
                    review = re.sub("(</span>)", "", str(review))
                    review = re.sub("(<span)", "", str(review))
                    reviewer = str(user[z]).split("itemprop=\"author\">")[1]
                    reviewer = reviewer.split("</span")[0]
                        
                    #insert reviews into mysql       
                    try:
                        data_review = (product_id, review, reviewer)
                        cursor.execute(add_review, data_review)
                    except:
                        pass
            
            #commit entry    
            cnx.commit()
            print('commit')
        
        except:
            #close mysql server
            cnx.close()
            #connect to mysql server
            cnx = mysql.connector.connect(host='152.19.68.141', user='ctolson', password='ilaYOU5!', database='sephora_cosmetics')
            cursor = cnx.cursor()
            #continue
            continue
        

blush
0
1
2
3
4
5
commit
6
commit
7
8
9
10
11
12
13
commit
14
15
16
17
18
commit
19
20
21
commit
22
commit
23
commit
24
commit
25
26
commit
27
28
29
30
31
32
commit
33
commit
34
commit
35
commit
36
commit
37
commit
38
39
commit
40
41
42
commit
43
44
commit
45
commit
46
47
commit
48
49
50
51
commit
52
53
54
commit
55
56
57
commit
58
59
commit
60
commit
61
commit
62
commit
63
64
65
66
67
68
commit
69
commit
70
71
72
commit
73
74
commit
75
76
commit
77
commit
78
79
commit
80
81
commit
82
83
commit
84
85
commit
86
commit
87
commit
88
89
90
91
commit
92
commit
93
94
95
commit
96
commit
97
98
99
commit
100
101
102
commit
103
104
commit
105
commit
106
107
108
commit
109
110
commit
111
112
commit
113
commit
114
115
commit
116
commit
117
commit
118
119
120
commit
121
122
commit
123
commit
124
125
126
127
128
commit
129
130
131
commit
132
commit
133
commit
134
135
136
commit
137
commit
138
commit
conditioner-hair
0
commit
1
commit
2
commit
3
commit
4
commit
5
commit
6
commit
7
commit
8
commit
9


In [4]:
#close mysql server
cnx.close()
